# Milestone 1
Abdul Haris

# Pendahuluan
Ingin eksplorasi data berita kejadian di Indonesia menggunakan dataset berita GDELT.

# Eksplorasi Data
## Download
Data ini bisa diproses melalui cloud atau lokal. Dalam project ini, proses data akan dilakukan secara lokal sehingga perlu didownload data GDELT terlebih dahulu.

Terdapat beberapa cara untuk mendownload data GDELT:
1. Menggunakan script
2. Menggunakan layanan GKG Record Exporter
3. Menggunakan layanan Event Record Exporter


## Buka Koneksi MySQL lokal
Silakan buka program SQL favorit anda (HeidiSQL, Laragon), lalu buat database dengan nama 'news'.

In [72]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

host = 'localhost'
port = '3306'
username = 'root'
password = ''
database = 'news'
# database = 'classicmodels'

# Create Connection to Database
engine = create_engine('mysql+pymysql://'+username+':'+password+'@'+host+':'+port+'/'+database)
# '''engine = create_engine('mysql+pymysql://root: @localhost:3306/academic')'''

def run(sql):
    df = pd.read_sql_query(sql, engine)
    return df

## 2. Menggunakan layanan GKG Record Exporter
The GKG Record Exporter allows you to rapidly export small subsets of data from the GDELT Global Knowledge Graph (GKG) that match your search criteria. You simply specify a set of person or organization names, locations, or Global Knowledge Graph Themes, along with an optional date range, and the system will automatically search the entire Global Knowledge Graph for all matching entries and output the list of matching GKG records and a list of all source articles providing the matching records. While you will still need programming experience to be able to use these exported results further, this tool allows you to rapidly export small subsets of data from the complete massive GKG database. Currently searches will only return the first 200,000 matching results. Your results will be emailed to you when complete, usually within 10 minutes, depending on server load.

### Download
Disini saya mendownload data GKG Record dengan rentang tanggal 1 Januari 2018 sampai 31 Oktober 2018 dengan keyword pencarian diskusi "Indonesia".

In [68]:
from os import listdir
from os.path import isfile, join
folder_id = './Data/'
files = [f for f in listdir(folder_id) if isfile(join(folder_id, f))]
print(files)

['20181106005702.5386.gkg.txt', 'Credit.csv']


### Check The Data

In [69]:
from os import listdir
from os.path import isfile, join
folder_id = './Data/'
files = [f for f in listdir(folder_id) if isfile(join(folder_id, f))]
print(files[0])

20181106005702.5386.gkg.txt


In [70]:
# Open first file for sanity check
df_awal = pd.read_csv(folder_id + files[0],sep="\t")

In [10]:
df_awal.head()

,Date,NumArticles,Counts,Themes,Locations,Persons,Organizations,ToneData,CAMEOEvents,Sources,SourceURLs
0,20180101,1,NaN,TAX_ETHNICITY;TAX_ETHNICITY_INDONESIAN;TAX_WOR...,"1#Indonesia#ID#ID#-5#120#ID;4#Jakarta, Jakarta...",anies baswedan;muhammad nasir,NaN,"6.51340996168582,7.66283524904215,1.1494252873...",719036954,coconuts.co,https://coconuts.co/jakarta/news/mass-wedding-...
1,20180101,1,NaN,TAX_RELIGION;TAX_RELIGION_SHARIA;UNGP_FORESTS_...,1#Indonesia#ID#ID#-5#120#ID,ahmad soekro tratmono,NaN,"0.315457413249211,1.89274447949527,1.577287066...",NaN,indonesia-investments.com,https://www.indonesia-investments.com/news/tod...
2,20180101,1,NaN,ECON_TAXATION;USPEC_POLICY1;EPU_POLICY;EPU_POL...,"1#Indonesia#ID#ID#-5#120#ID;4#Jakarta, Jakarta...",NaN,freeport mcmoran inc,"-2.04918032786885,1.63934426229508,3.688524590...",NaN,customstoday.com.pk,http://www.customstoday.com.pk/despite-tax-sho...
3,20180101,1,NaN,TAX_FNCACT;TAX_FNCACT_MINISTER;LEADER;TAX_FNCA...,"4#Tuas, Singapore (General), Singapore#SN#SN00...",donald trump;halimah yacob,nanyang girl high school;singapore malayalee a...,"1.5647226173542,3.62731152204836,2.06258890469...","719030357,719031298,719031301",straitstimes.com,http://www.straitstimes.com/singapore/a-strong...
4,20180101,1,"AFFECT#51##4#Kalimantan, Indonesia (General), ...",NATURAL_DISASTER;NATURAL_DISASTER_DROWN;MARITI...,"4#Tarakan, Kalimantan Timur, Indonesia#ID#ID14...",NaN,NaN,"-2.9940119760479,2.09580838323353,5.0898203592...","719080657,719142399",dailymail.co.uk,http://www.dailymail.co.uk/news/article-522596...


In [21]:
df_awal.shape

(200001, 11)

Berikut penjelasan header dari data GKG yang didapat:
![title](gkg.png)

In [145]:
# for i in df_awal["Themes"]:
#     print(i)
ukuran = df_awal["SourceURLs"].str.len().max()
print(ukuran)

38165


Create table "gdelt_gkg" terlebih dahulu di database jika belum ada.

In [73]:
engine.execute("""
CREATE TABLE IF NOT EXISTS 
gdelt_gkg (Date INT,
	NumArticles INT,
	Counts TEXT,
	Themes TEXT,
	Locations TEXT,
	Persons TEXT,
	Organizations TEXT,
	ToneData VARCHAR(256),
	CAMEOEvents TEXT,
	Sources TEXT,
	SourceURLs TEXT )
""")

## Simpan ke MySQL

In [169]:
df_awal.columns
df_awal.to_sql(name='gdelt_gkg', con = engine, if_exists='append', index=False)

## Tranformasi Data

### Cek Awal
Setelah data tersimpan secara sukses di MySQL, kita melakukan eksplorasi sederhana. Langkah pertama yang perlu dilakukan adalah melakukan filter artikel yang benar-benar sesuai dengan Indonesia dengan sumber media asing. Dengan asumsi judul berita dimuat dalam URL artikel, maka dilakukan filter kolom 'SOURCEURL' dengan menggunakan kata kunci yang berhubungan dengan topik Indonesia. Sejauh ini ada beberapa kata kunci adalah sebagai berikut:

- Indonesia
- Java
- Jakarta
- Bali
- Lombok

In [182]:
# SQL query to look for keyword in SOURCEURL
data_indonesia = run("""
SELECT 
    * 
FROM 
    gdelt_gkg
WHERE
    SourceURLs LIKE '%%indonesia%%' OR
    SourceURLs LIKE '%%java%%' OR
    SourceURLs LIKE '%%bali%%' OR    
    SourceURLs LIKE '%%lombok%%' OR  
    SourceURLs LIKE '%%jakarta%%'
    """)

In [184]:
print(data_indonesia.shape)
data_indonesia.sample(25)

(43503, 11)


,Date,NumArticles,Counts,Themes,Locations,Persons,Organizations,ToneData,CAMEOEvents,Sources,SourceURLs
29112,20180427,1,None,LEADER;TAX_FNCACT;TAX_FNCACT_PRESIDENT;USPEC_P...,"1#Vietnam, Republic Of#VM#VM#16.166667#107.833...",nguyen xuan phuc;alan peter s cayetano;mar yap...,department of foreign affairs;association of s...,"0.213675213675214,1.70940170940171,1.495726495...","751280109,751150930,751136186,751209646,751280...",bworldonline.com,http://bworldonline.com/duterte-to-hold-bilate...
21794,20180331,1,None,NATURAL_DISASTER;NATURAL_DISASTER_EARTHQUAKE;C...,1#Indonesia#ID#ID#-5#120#ID,None,earthquake notification service,"-0.886262924667651,0.295420974889217,1.1816838...",None,earthquakenewstoday.com,http://www.earthquakenewstoday.com/2018/03/31/...
15199,20180301,1,"SEIZE#200#m yacht#3#Hollywood, California, Uni...",MARITIME;SEIZE;CORRUPTION;WB_2019_ANTI_CORRUPT...,1#Australia#AS#AS#-25#135#AS;1#Malaysia#MY#MY#...,winston churchill;miranda kerr;lindsay lohan;m...,None,"-0.48076923076923,3.84615384615385,4.326923076...","734990881,734990910,734990911,734991563,734991...",express.co.uk,https://www.express.co.uk/news/uk/925704/Yacht...
3889,20180117,1,KILL#60##1#Indonesia#ID#ID#-5#120#ID;,TAX_ETHNICITY;TAX_ETHNICITY_INDONESIAN;TAX_WOR...,1#Indonesia#ID#ID#-5#120#ID,None,None,"-3.33333333333333,2.22222222222222,5.555555555...",None,aljazeera.com,http://www.aljazeera.com/news/2018/01/indonesi...
32314,20180513,1,KILL#9##1#Indonesia#ID#ID#-5#120#ID;WOUND#24##...,SOC_SUICIDE;CRISISLEX_CRISISLEXREC;TERROR;SUIC...,"4#Diponegoro, Jawa Timur, Indonesia#ID#ID08#-7...",frans barung mangera,santa maria roman catholic church;pantekosta c...,"-10.625,0.625,11.25,11.875,23.4375,0","755487937,755474126,755480659,755468413,755480...",vancourier.com,http://www.vancourier.com/9-dead-40-wounded-as...
31218,20180509,1,None,TAX_ETHNICITY;TAX_ETHNICITY_DUTCH;TAX_WORLDLAN...,"1#Japan#JA#JA#36#138#JA;4#Merdeka, Nusa Tengga...",lucas martin sarkies;surabayahotel majapahit,hotel freedom hotel majapahit;raffles hotel;ho...,"0,3.24574961360124,3.24574961360124,6.49149922...",None,indonesiaexpat.biz,http://indonesiaexpat.biz/travel/hotel-majapah...
12218,20180219,1,None,WB_678_DIGITAL_GOVERNMENT;WB_694_BROADCAST_AND...,"1#Indonesia#ID#ID#-5#120#ID;4#Jakarta, Jakarta...",nick macfie;tabita diela,google;facebook;facebook inc whatsapp messenge...,"-3.94088669950739,0.164203612479475,4.10509031...",None,yahoo.com,https://au.news.yahoo.com/a/39222774/new-indon...
25108,20180412,1,None,LEADER;,"4#Denpasar, Bali, Indonesia#ID#ID02#-8.65#115....",jusuf kalla,bali nusa dua convention center;international ...,"2.29357798165138,2.29357798165138,0,2.29357798...","746858359,746858361,746858370,746858371,746859...",tralac.org,https://www.tralac.org/news/article/12926-indo...
39846,20180601,1,None,MANMADE_DISASTER_IMPLIED;,"1#Australia#AS#AS#-25#135#AS;4#Lombok Strait, ...",michel bourez;phil toledo;steph gilmore;griff ...,None,"0,1.75695461200586,1.75695461200586,3.51390922...",760947153,surfer.com,https://www.surfer.com/features/bali-pro-indo-...
35632,20180516,1,None,SOC_POINTSOFINTEREST;SOC_POINTSOFINTEREST_AIRP...,"4#Halim Perdanakusuma, Jakarta Raya, Indonesia...",majalengka-westjava kertajati;bandara internas...,mutia fauzia kertajati international airport i...,"1.03806228373702,1.03806228373702,0,1.03806228...",None,thejakartapost.com,http://www.thejakartapost.com/travel/2018/05/1...


In [167]:
df_source = df_awal.dropna(subset=['Sources'])
list_link = []

# f = open('Data\Sources.txt','w')

for index, row in df_source.iterrows():
    splitted = row['Sources'].split(";")
    for i in splitted:
#         f.write(i)
        list_link.append(i)
#     URL = row['SourceURLs'].split('<UDIV>')
    
data_link = pd.Series(list_link)

# Count by its occurences
data_link_count = data_link.value_counts()
data_link_count
# f.close()

thejakartapost.com               6491
reuters.com                      4781
en.tempo.co                      4121
ianslive.in                      2755
msn.com                          2746
openpr.com                       2409
yahoo.com                        2222
tvguide.co.uk                    2208
4-traders.com                    2167
philstar.com                     2098
nzcity.co.nz                     1854
antaranews.com                   1792
indiatimes.com                   1708
business-standard.com            1696
republika.co.id                  1551
jakartaglobe.id                  1548
thestar.com.my                   1391
straitstimes.com                 1365
businesstimes.com.sg             1363
theedgemarkets.com               1332
themalaymailonline.com           1308
dailymail.co.uk                  1305
netralnews.com                   1049
btvi.in                           945
iheart.com                        936
bangkokpost.com                   920
military-tec

In [166]:
# save ke dalam CSV file
data_link_count.to_csv("Data\Sources.csv")